# Example fcst Nixtla
Se utiliza el fcst del template del package nixtla porque es bastante simple de realizar ya que se entrena solo un modelo para hacer todos los forecasts y no realiza muchas transformaciones a los datos

Fuente: SOURCE: https://www.kaggle.com/code/lemuz90/m5-mlforecast-eval

In [1]:
import pandas as pd
import pickle
import numpy as np

In [2]:
import os
# set path root of repo
actual_path = os.path.abspath(os.getcwd())
list_root_path = actual_path.split('/')[:-1]
root_path = '/'.join(list_root_path)
os.chdir(root_path)
print('root path: ', root_path)

root path:  /Users/joseortega/Documents/GitHub/forecasting-m5-dataset


### 1. Read raw files dtype corrrect

In [3]:
folder_data = 'data/data_input_dtype/'
df_cal = pd.read_pickle(folder_data + 'calendar.pkl')
df_sales = pd.read_pickle(folder_data + 'sales_train_evaluation.pkl')
df_price = pd.read_pickle(folder_data + 'sell_prices.pkl')
df_sample_output = pd.read_pickle(folder_data + 'sample_submission.pkl')

In [4]:
df_cal.head(3)

,date,wm_yr_wk,weekday,wday,month,year,d,event_name_1,event_type_1,event_name_2,event_type_2,snap_CA,snap_TX,snap_WI
0,2011-01-29,11101,Saturday,1,1,2011,d_1,nan,nan,nan,nan,0,0,0
1,2011-01-30,11101,Sunday,2,1,2011,d_2,nan,nan,nan,nan,0,0,0
2,2011-01-31,11101,Monday,3,1,2011,d_3,nan,nan,nan,nan,0,0,0


In [5]:
df_sales.head(3)

,id,item_id,dept_id,cat_id,store_id,state_id,d_1,d_2,d_3,d_4,...,d_1932,d_1933,d_1934,d_1935,d_1936,d_1937,d_1938,d_1939,d_1940,d_1941
0,HOBBIES_1_001_CA_1_evaluation,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_1,CA,0.0,0.0,0.0,0.0,...,2.0,4.0,0.0,0.0,0.0,0.0,3.0,3.0,0.0,1.0
1,HOBBIES_1_002_CA_1_evaluation,HOBBIES_1_002,HOBBIES_1,HOBBIES,CA_1,CA,0.0,0.0,0.0,0.0,...,0.0,1.0,2.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0
2,HOBBIES_1_003_CA_1_evaluation,HOBBIES_1_003,HOBBIES_1,HOBBIES,CA_1,CA,0.0,0.0,0.0,0.0,...,1.0,0.0,2.0,0.0,0.0,0.0,2.0,3.0,0.0,1.0


In [6]:
df_price.head(3)

,store_id,item_id,wm_yr_wk,sell_price
0,CA_1,HOBBIES_1_001,11325,9.58
1,CA_1,HOBBIES_1_001,11326,9.58
2,CA_1,HOBBIES_1_001,11327,8.26


In [7]:
df_sample_output.head(3)

,id,F1,F2,F3,F4,F5,F6,F7,F8,F9,...,F19,F20,F21,F22,F23,F24,F25,F26,F27,F28
0,HOBBIES_1_001_CA_1_validation,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,HOBBIES_1_002_CA_1_validation,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,HOBBIES_1_003_CA_1_validation,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


### 2. Transformar datos a formato input de fcst nixtla (melt)

In [8]:
# transformar las ventas a formato que necesita nixtla. Todas unidades en una columna
# MELT dataframe ID_SERIE, TIMESTAMP, VALUE
df_sales_long = df_sales.melt(
    id_vars=['id', 'item_id', 'dept_id', 'cat_id', 'store_id', 'state_id'],
    var_name='d',
    value_name='y'
)

### 3. Append calendario con datos ventas

In [9]:
print(df_sales_long.shape[0]) # tamaño original

59181090


In [10]:
# eliminar columnas de df calender
df_cal.drop(columns = ['weekday', 'wday', 'month', 'year'], inplace = True)

In [11]:
# columa día transformar a formato de columna día del dataframe calendario y luego hacer merge de ambos df
df_sales_long['d'] = df_sales_long['d'].astype(df_cal.d.dtype)
df_sales_long = df_sales_long.merge(df_cal, on=['d'])

In [23]:
# obtener una lista con timestamp de todas las fechas en los datos. Hasta día 1941
dates = sorted(df_sales_long['date'].unique())
dates[0:5]

[Timestamp('2011-01-29 00:00:00'),
 Timestamp('2011-01-30 00:00:00'),
 Timestamp('2011-01-31 00:00:00'),
 Timestamp('2011-02-01 00:00:00'),
 Timestamp('2011-02-02 00:00:00')]

In [24]:
# ordenar dataframe
df_sales_long = df_sales_long.sort_values(['id', 'date'])

show informativo

In [26]:
# show informativo
df_sales_long.head(3)

,id,item_id,dept_id,cat_id,store_id,state_id,d,y,date,wm_yr_wk,event_name_1,event_type_1,event_name_2,event_type_2,snap_CA,snap_TX,snap_WI
0,HOBBIES_1_001_CA_1_evaluation,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_1,CA,d_1,0.0,2011-01-29,11101,nan,nan,nan,nan,0,0,0
30490,HOBBIES_1_001_CA_1_evaluation,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_1,CA,d_2,0.0,2011-01-30,11101,nan,nan,nan,nan,0,0,0
60980,HOBBIES_1_001_CA_1_evaluation,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_1,CA,d_3,0.0,2011-01-31,11101,nan,nan,nan,nan,0,0,0


In [27]:
# show informativo nulos datos
df_sales_long.isnull().sum()

id              0
item_id         0
dept_id         0
cat_id          0
store_id        0
state_id        0
d               0
y               0
date            0
wm_yr_wk        0
event_name_1    0
event_type_1    0
event_name_2    0
event_type_2    0
snap_CA         0
snap_TX         0
snap_WI         0
dtype: int64

In [28]:
# show informativo fecha maxima de dia -> d_1941
df_sales_long['d']#.as_ordered()

0              d_1
30490          d_2
60980          d_3
91470          d_4
121960         d_5
             ...  
59059129    d_1937
59089619    d_1938
59120109    d_1939
59150599    d_1940
59181089    d_1941
Name: d, Length: 59181090, dtype: category
Categories (1969, object): ['d_1', 'd_10', 'd_100', 'd_1000', ..., 'd_996', 'd_997', 'd_998', 'd_999']

### 4. Transformaciones

In [31]:
without_leading_zeros = df_sales_long['y'].gt(0).groupby(df_sales_long['id'], observed=True).transform('cummax')

In [37]:
df_sales_long['y']

0           0.0
30490       0.0
60980       0.0
91470       0.0
121960      0.0
           ... 
59059129    0.0
59089619    2.0
59120109    2.0
59150599    5.0
59181089    1.0
Name: y, Length: 59181090, dtype: float32

In [36]:
df_sales_long['y'].gt(0)

0           False
30490       False
60980       False
91470       False
121960      False
            ...  
59059129    False
59089619     True
59120109     True
59150599     True
59181089     True
Name: y, Length: 59181090, dtype: bool

In [ ]:







above_min_date = long['date'] >= dates[-400]
keep_mask = without_leading_zeros & above_min_date
long = long[keep_mask]

print(long.shape[0])